In [37]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from finta import TA
from pandas.tseries.offsets import DateOffset
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout

%matplotlib inline

In [48]:
# Import indicator dataframe
# Start with 1-minute... alghough we would like to attatch them all together soon.
df = pd.read_csv("../algotrader/resources/aapl_1min_pivot_point_indicator_df.csv")
df.head()

# df = pd.read_csv("../algotrader/resources/aapl_3min_pivot_point_indicator_df.csv")
# df.head()

# df = pd.read_csv("../algotrader/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()



,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
0,2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0
1,2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0
2,2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0
3,2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0
4,2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0


In [49]:
# our df does not have date-time
hello = df['timestamp']
hello

0         2023-01-03 15:04:00+00:00
1         2023-01-03 15:05:00+00:00
2         2023-01-03 15:06:00+00:00
3         2023-01-03 15:07:00+00:00
4         2023-01-03 15:08:00+00:00
                    ...            
153677    2024-01-17 00:47:00+00:00
153678    2024-01-17 00:52:00+00:00
153679    2024-01-17 00:55:00+00:00
153680    2024-01-17 00:57:00+00:00
153681    2024-01-17 00:58:00+00:00
Name: timestamp, Length: 153682, dtype: object

In [50]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])

hello = df['timestamp']
hello

0        2023-01-03 15:04:00+00:00
1        2023-01-03 15:05:00+00:00
2        2023-01-03 15:06:00+00:00
3        2023-01-03 15:07:00+00:00
4        2023-01-03 15:08:00+00:00
                    ...           
153677   2024-01-17 00:47:00+00:00
153678   2024-01-17 00:52:00+00:00
153679   2024-01-17 00:55:00+00:00
153680   2024-01-17 00:57:00+00:00
153681   2024-01-17 00:58:00+00:00
Name: timestamp, Length: 153682, dtype: datetime64[ns, UTC]

In [51]:
df.set_index('timestamp', inplace=True)

df.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0


In [52]:
# X is everything except the signal column
X = df.drop('signal', axis=1)

X.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CCI,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-116.423590,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-101.126762,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-99.309857,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-88.964539,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-84.000283,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0


In [53]:
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()

X.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CCI,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:05:00+00:00,126.5001,126.880,126.4166,5733.0,126.820,513722.0,126.624892,-0.002516,126.87386,127.15340,...,-116.423590,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0
2023-01-03 15:06:00+00:00,126.8500,126.895,126.4400,3763.0,126.505,362900.0,126.625878,0.002766,126.80786,127.06139,...,-101.126762,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0
2023-01-03 15:07:00+00:00,126.5800,126.870,126.5300,2930.0,126.860,240388.0,126.664497,-0.002128,126.71786,126.94739,...,-99.309857,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0
2023-01-03 15:08:00+00:00,126.7100,126.870,126.5000,3189.0,126.580,289721.0,126.666956,0.001027,126.69186,126.87129,...,-88.964539,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0
2023-01-03 15:09:00+00:00,126.7066,126.760,126.5601,2861.0,126.705,255144.0,126.687038,-0.000027,126.66934,126.81495,...,-84.000283,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0


We may look to consider what it would do if we changed the amount that we shifted by. Perhaps we tried predicting 5 minutes into the future... how about an hour?

In [54]:
# y is the signal column
y = df[("signal")]

y


timestamp
2023-01-03 15:04:00+00:00    0
2023-01-03 15:05:00+00:00    0
2023-01-03 15:06:00+00:00    0
2023-01-03 15:07:00+00:00    0
2023-01-03 15:08:00+00:00    0
                            ..
2024-01-17 00:47:00+00:00    0
2024-01-17 00:52:00+00:00    0
2024-01-17 00:55:00+00:00    0
2024-01-17 00:57:00+00:00    0
2024-01-17 00:58:00+00:00    0
Name: signal, Length: 153682, dtype: int64

In [56]:
# Set start of training period
training_begin = X.index.min()

print(training_begin)

2023-01-03 15:05:00+00:00


In [61]:
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)

print(training_end)

2023-10-03 15:05:00+00:00


In [62]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [63]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
display(X_train_scaled)
display(X_test_scaled)

array([[-2.35124626, -2.33365604, -2.35202174, ..., -1.80656103,
         7.44654984, -1.12513333],
       [-2.33149852, -2.33280927, -2.3507014 , ..., -1.7587487 ,
         6.88368505, -1.12513333],
       [-2.34673685, -2.33422056, -2.3456232 , ..., -1.77349419,
         6.38717519, -1.12513333],
       ...,
       [ 0.18395335,  0.19028715,  0.18557785, ..., -0.68450998,
        -0.27235233,  1.14753163],
       [ 0.18762183,  0.18831135,  0.18558349, ..., -0.69605814,
        -0.27994579,  1.14753163],
       [ 0.18705745,  0.1926694 ,  0.18727058, ..., -0.67466796,
        -0.40165214,  1.14753163]])

array([[ 0.18705745,  0.1926694 ,  0.18727058, ..., -0.67466796,
        -0.40165214,  1.14753163],
       [ 0.19186034,  0.19028715,  0.19009181, ..., -0.64040263,
        -0.5609811 ,  1.14753163],
       [ 0.18827087,  0.19141618,  0.18896332, ..., -0.64296252,
        -0.64949377,  1.14753163],
       ...,
       [ 0.84032995,  0.84004305,  0.843487  , ...,  1.54445325,
        -0.47423821, -1.12513333],
       [ 0.83750804,  0.83609145,  0.84066577, ...,  0.16007704,
        -0.41855528, -1.12513333],
       [ 0.83807242,  0.83665596,  0.84123002, ..., -0.20120459,
        -0.49805655, -1.12513333]])

### Now that we have scaled our data, we can build our neural network.

In [ ]:
num_predictors = len(X.columns)

# We have a binary outcome, so one output
num_classes = 1

num_predictors

In [ ]:
nn_model = Sequential()

In [ ]:
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))

In [ ]:
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))

# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))

In [ ]:
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))

In [ ]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

In [ ]:
# Fit model
num_epochs = 50

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

In [ ]:
# Deployment (Hypothetical)
while True:
    current_data = your_trading_api.get_real_time_data()
    current_data_processed = preprocess_data(current_data)
    prediction = model.predict(current_data_processed)
    if prediction > some_threshold:
        your_trading_api.execute_trade()

# Placeholder Functions
def combine_data(historical, news):
    # Combine and return data
    pass

def split_data(data):
    # Split and return data
    pass

def backtest_strategy(model, data):
    # Implement backtesting logic
    pass

def preprocess_data(data):
    # Data preprocessing steps
    pass

In [ ]:
# ChatGPT 3.5 starter code for a trading bot: 

import alpaca_trade_api as tradeapi
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import time


def get_minute_data(symbol, start, end):
    barset = api.get_barset(symbol, '1Min', start=start, end=end).df
    return barset[symbol]

def generate_features(data):
    # Calculate simple moving averages
    data['short_mavg'] = data['close'].rolling(window=10).mean()
    data['long_mavg'] = data['close'].rolling(window=30).mean()

    # Calculate price and volume differences
    data['price_diff'] = data['close'].diff()
    data['volume_diff'] = data['volume'].diff()

    # Create binary labels for entry signals (1: Buy, -1: Sell, 0: Hold)
    data['entry_signal'] = np.where(data['short_mavg'] > data['long_mavg'], 1, -1)
    data['entry_signal'] = np.where(data['entry_signal'].shift(1) == 1, 0, data['entry_signal'])

    # Create binary labels for exit signals (1: Hold, -1: Sell, 0: Exit)
    data['exit_signal'] = np.where(data['short_mavg'] > data['long_mavg'], 1, -1)
    data['exit_signal'] = np.where(data['exit_signal'].shift(1) == -1, 0, data['exit_signal'])

    return data[['short_mavg', 'long_mavg', 'price_diff', 'volume_diff']], data['entry_signal'], data['exit_signal']

def train_model(features, entry_labels, exit_labels):
    # Train separate models for entry and exit signals
    entry_model = RandomForestClassifier(n_estimators=100, random_state=42)
    entry_model.fit(features, entry_labels)

    exit_model = RandomForestClassifier(n_estimators=100, random_state=42)
    exit_model.fit(features, exit_labels)

    return entry_model, exit_model

def execute_trades(api, symbol, entry_model, exit_model, features):
    entry_signals = entry_model.predict(features)
    exit_signals = exit_model.predict(features)

    for i in range(1, len(entry_signals)):
        current_entry_signal = entry_signals[i]
        previous_entry_signal = entry_signals[i - 1]

        current_exit_signal = exit_signals[i]
        previous_exit_signal = exit_signals[i - 1]

        if current_entry_signal == 1 and previous_entry_signal == 0:
            api.submit_order(
                symbol=symbol,
                qty=1,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
        elif current_entry_signal == -1 and previous_entry_signal == 0:
            api.submit_order(
                symbol=symbol,
                qty=1,
                side='sell',
                type='market',
                time_in_force='gtc'
            )

        if current_exit_signal == -1 and previous_exit_signal == 1:
            api.submit_order(
                symbol=symbol,
                qty=1,
                side='sell',
                type='market',
                time_in_force='gtc'
            )

if __name__ == "__main__":
    symbol = 'AAPL'
    start_date = '2023-01-01'
    end_date = '2023-01-02'

    while True:
        current_time = pd.Timestamp.now(tz='America/New_York')
        
        if current_time.time() >= pd.Timestamp('09:30').time() and current_time.time() <= pd.Timestamp('15:59').time():
            # Market is open
            minute_data = get_minute_data(symbol, start_date, end_date)
            features, entry_labels, exit_labels = generate_features(minute_data)
            
            # Drop NaN values introduced by rolling averages
            features = features.dropna()
            entry_labels = entry_labels.loc[features.index]
            exit_labels = exit_labels.loc[features.index]

            # Split data into training and testing sets
            features_train, features_test, entry_labels_train, entry_labels_test, exit_labels_train, exit_labels_test = train_test_split(
                features, entry_labels, exit_labels, test_size=0.2, shuffle=False
            )

            # Train the machine learning models
            entry_model, exit_model = train_model(features_train, entry_labels_train, exit_labels_train)

            # Make predictions on the test set
            entry_predictions = entry_model.predict(features_test)
            exit_predictions = exit_model.predict(features_test)

            # Evaluate the models
            entry_accuracy = accuracy_score(entry_labels_test, entry_predictions)
            exit_accuracy = accuracy_score(exit_labels_test, exit_predictions)
            print(f'Entry Model Accuracy: {entry_accuracy}, Exit Model Accuracy: {exit_accuracy}')

            # Execute trades based on predictions
            execute_trades(api, symbol, entry_model, exit_model, features)

            # Sleep for a minute to avoid excessive API calls
            time.sleep(60)
        else:
            # Market is closed, wait for it to open
            time.sleep(60)
